# OpenAI Whisper 을 이용한 음성인식 실습

Whisper를 이용하여 음성인식을 수행합니다. 본 실습에서는 음성인식 모델의 훈련은 포함하지 않습니다.
OpenAI에서 제공하는 음성인식 모델을 이용하여 평가데이터에 대하여 인식을 수행하고 인식률을 측정하는 방법을 설명합니다.

## 환경 설정
먼저 실습환경을 설정합니다. 아래 스크립트는 jupyter 환경을 기준으로 설명합니다.
일반 리눅스 환경에서는 먼저 https://github.com/pkyoung/a1003.git 를 내려받고
jupyter notebook을 실행하여 이 파일을 jupyter환경에서 열어봅니다.
```
git clone https://github.com/pkyoung/a1003.git ~/a1003
conda create -n a1003 pythoon=3.10
conda activate a1003
pip install jupyter
jupyter notebook --notebookdir ~/a1003/local
```

- jupyter 환경이 아닌 일반 shell 환경에서도 실행가능합니다. 이 경우
conda 환경 안에서 `!` 로 시작하는 명령어들을 순차적으로 실행합니다.
`!`는 입력하지 않습니다.
-  작업 디렉토리에 a1003 디렉토리가 없으면, https://github.com/pkyoung/a1003.git 의 코드를 작업 디렉토리에 내려받습니다. Colab 환경에서는 작업 디렉토리는 /content 입니다. 그 밖의 linux 환경에서는 작업디렉토리는 홈디렉토리이고 `~/`가 홈디렉토리를 의미합니다.
- 실습은 a1003/local 디렉토리에서 수행합니다. 이 디렉토리로 이동합니다.

In [ ]:
# Run this cell to automatically clone the repo
import sys
import os

if 'google.colab' in sys.modules:
    if not os.path.exists("/content/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" /content/a1003
    %cd /content/a1003/local
else:
    if not os.path.exists("~/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" ~/a1003
    %cd ~/a1003/local

## OpenAI Whisper 설치

Whisper는 아래와 같이 `openai-whisper` 패키지를 설치하면 됩니다. 여러가지 의존성 패키지가 설치되기때문에 시간이 소요됩니다.

In [ ]:
pip install openai-whisper

## 평가용 데이터셋 다운로드

huggingface에서 테스트용 데이터셋을 내려받습니다.
9명의 화자가 20개의 짧은 문장을 발성한 wav 파일과 정답전사문 파일입니다.


In [ ]:
if not os.path.exists("data/a1003"):
  !git clone https://hf.co/datasets/pkyoung/a1003.git ./data/a1003

내려받은 파일을 확인해봅니다.
내려받은 디렉토리의 구조는 다음과 같습니다.
```
data/a1003
    ├── README.md
    ├── result_espnet_ma16k2401a.txt
    ├── result_whisper_turbo.txt
    ├── text
    ├── wav.scp
    ├── wav
    │   ├── spk1
    │   │   ├── 1.wav
    │   │   ├── 10.wav
    │   │   ├── 11.wav
    ...
```
`text` 파일과 `wav.scp` 파일은 앞서 설명한 `kaldi` 파일과 같은 형식을 갖습니다. 파일은 다음과 같은 방법으로 들어볼 수 있습니다.

In [ ]:
from IPython.display import Audio
filename = "data/a1003/wav/spk1/1.wav"
Audio(filename, autoplay=False)

## Whipser 음성인식

한개의 wav 파일에 대해서 인식을 수행해봅니다. 간단하게 파일을 입력으로 `whisper` 명령어를 실행합니다.

In [ ]:
!whisper data/a1003/wav/spk1/1.wav

whisper는 task로 transcribe와 translate 두가지를 할 수 있습니다. 인식에 사용할 모델, 인식대상 언어를 지정하여 음성인식을 수행합니다. (Colab 환경에서 약 5분 이상 소요)

In [ ]:
!whisper --model large-v3 --language ko --task transcribe data/a1003/wav/spk6/recording_17.wav

파이선 코드 안에서는 다음과 같은 방법으로 이용할 수 있습니다. Low-level 핸들링을 위해서는 `transcribe()` 함수 외에 `decode()` 함수를 사용할 수 있습니다.

In [ ]:
import whisper
model = whisper.load_model("tiny")
result = model.transcribe("data/a1003/wav/spk6/recording_17.wav")
print(result["text"])

전체 테스트 데이터셋 (180개) 에 대해서 인식을 수행해서 결과를 `result.txt` 파일에 저장합니다.

보통의 GPU 카드가 설치된 머신에서는 수 분 이내로 인식이 완료됩니다. Colab 환경에서는 10분 이상 소요될 수 있습니다.

In [ ]:
import whisper
import tqdm
import warnings
warnings.filterwarnings("ignore")

model = whisper.load_model("tiny")

with open('data/a1003/wav.scp') as fscp:
    lines = fscp.readlines()

with open("result_whisper.txt", "w") as f:
  for line in  tqdm.tqdm(lines, desc="Transcribing", unit="utterance"):
    uttid, filepath = line.split()
    result = model.transcribe(filepath)
    print(uttid, result["text"], file=f)

저장된 인식결과를 확인합니다.

In [ ]:
!head result_whisper.txt

## Whisper 모델의 음성인식 오류율 측정
발화별 CER과 평균 CER을 계산합니다.

`data/a1003/result_whisper_turbo.txt` 파일에 `large-v3-turbo` 모델의
인식 결과를 미리 만들어 두었습니다. 이 파일을 사용해서 인식률을 측정합니다.

In [ ]:
if (
    os.path.exists('result.txt') and
    sum(1 for _ in open('result_whisper.txt')) == sum(1 for _ in open('data/a1003/wav.scp'))
):
    !cp result_whisper.txt ./result.txt
else:
    !cp data/a1003/result_whisper_turbo.txt ./result.txt


In [ ]:
!pip install editdistance
!python uttcer.py data/a1003/text result.txt

화자별 음성인식 성능을 계산합니다.

In [ ]:
%%bash
for s in spk{1..9}; do
  echo -n "$s " >&2
  python uttcer.py data/a1003/text <(grep "$s/" result.txt) > /dev/null
done

평균 오류율이 높은 화자에 대해서는 몇몇 파일을 들어봅니다. 주석처리된 부분을 바꿔가면서 들어봅니다.

In [ ]:
from IPython.display import Audio
filename = "data/a1003/wav/spk2/utt1.wav"
#filename = "data/a1003/wav/spk7/recording_00.wav"
#filename = "data/a1003/wav/spk3/data15.wav"

Audio(filename, autoplay=False)

문장부호를 제외한 CER을 계산합니다.

In [ ]:
!python uttwer.py <(python pp.py -s data/a1003/text) <(python pp.py -s result.txt) > /dev/null

## Whisper 모델별 음성인식 성능 및 속도

각 모델별 모델크기, 인식 소요 시간 및 인식 성능은 다음 표와 같습니다.
인식소요시간 및 인식성능은 환경에 따라 달라질 수 있습니다. (WER1, CER1: 문장부호 제외 전/WER2, CER2: 문장부호 제외 후)

| Model          | Size | Elapsed | RTF | WER1 | CER1 | WER2 | CER2 |
| ---            | ---  | ---     | --- | ---  | ---  | ---  | ---  |
| tiny           | 73M  | 1m6s    |0.04 |50.11 |26.62 |44.10 |24.55 |
| base           | 139M | 1m2s    |0.04 |38.94 |17.74 |32.76 |15.61 |
| small          | 462M | 1m38s   |0.06 |32.73 |13.07 |24.55 |10.13 |
| medium         | 1.5G | 3m5s    |0.11 |29.57 |11.14 |21.90 | 8.46 |
| large-v1       | 2.9G | 4m0s    |0.15 |25.33 | 9.03 |16.98 | 6.15 |
| large-v2       | 2.9G | 3m46s   |0.14 |30.34 |13.43 |21.64 |10.24 |
| large-v3       | 2.9G | 4m1s    |0.15 |29.44 |13.46 |20.40 |10.43 |
| large-v3-turbo | 1.6G | 1m19s   |0.05 |24.18 | 8.29 |14.67 | 5.05 |
